# Safety in Funflow

Besides the safety that comes with immutable data and strong, static typing in Haskell, Funflow considers common properties of data workflow/pipeline authorship and strives to offer some extra safety tailored to this domain. In particular, Funflow offers some ___configuration-time_ validation__.

What does this mean? Well, one upside of Funflow's ability to decouple pipeline _declaration_ from pipeline _execution_ is that we gain an additional layer of information, and a platform/opportunity to perform more informed validation of what will unfold at execution time. As such, we can consider the possibility for errors that could arise at arbitrary timepoints in the course of execution of a workflow.

Just as static typing allows for _compile-time_ checking, catching errors that could otherwise occur much later in a program's lifecycle (during _run-time_), this concrete decoupling of conceptual phases of a pipeline's lifecycle allows us to surface errors. We'd much rather know about and be able to fix an error before a pipeline ever starts than to run into the error after great cost (certainly of compute time, and perhaps also of compute money!)

In [1]:
import qualified Data.Map as Map
import qualified Data.Text as T
import System.Environment (lookupEnv, setEnv)

In [2]:
import qualified Data.CAS.ContentStore as CS
import Funflow.Config ( Configurable( ConfigFromEnv ) )
import Funflow.Flow (dockerFlow, Flow)
import Funflow.Run (runFlow)
import qualified Funflow.Tasks.Docker as DE

First, let's establish an environment variable that's not set.

In [3]:
tmpEnvVar = "FUNFLOW_TEST"
lookupEnv tmpEnvVar

Nothing

Now, to set up the demo of ___config-time_ failure__ (i.e., when the task is _interpreted_, but before it's executed), let's define a task configuration that uses the unset environment variable:

In [4]:
dockConf = DE.DockerTaskConfig {DE.image = T.pack "bash:latest", DE.command = T.pack "echo", DE.args = [DE.Arg $ ConfigFromEnv (T.pack tmpEnvVar)]}
:t dockConf

dockConf :: DockerTaskConfig

To arbitrarily scale reusability across a DAG, `Funflow` execution is framed in terms of _flows_ rather than _tasks_, so let's create a flow based on our (known) faulty configuration:

In [5]:
dockFlow :: Flow DE.DockerTaskInput CS.Item
dockFlow = dockerFlow dockConf
:t dockFlow

dockFlow :: forall (r :: RopeRec) (core :: BinEff) (m :: * -> *) (mantle :: [Strand]).
(RecElemFCtx r (Weaver core), MonadIO m, RecElem r ("docker" ::: DockerTask) ("docker" ::: DockerTask) mantle mantle (RIndex ("docker" ::: DockerTask) mantle),
 RecElem r ("simple" ::: SimpleTask) ("simple" ::: SimpleTask) mantle mantle (RIndex ("simple" ::: SimpleTask) mantle),
 RecElem r ("store" ::: StoreTask) ("store" ::: StoreTask) mantle mantle (RIndex ("store" ::: StoreTask) mantle), ArrowChoice core, ThrowEffect SomeException core, ThrowEffect StringException core,
 ThrowEffect DockerClientError core, TryEffect SomeException core, TryEffect StringException core, TryEffect DockerClientError core, SieveTrans m core, ProvidesCaching core,
 FieldType "simple" mantle ~ SimpleTask, FieldType "store" mantle ~ StoreTask, FieldType "docker" mantle ~ DockerTask) =>
Rope r mantle core DockerTaskInput Item

Now, let's execute the flow and observe the failure:

In [6]:
runFlow dockFlow DE.DockerTaskInput{ DE.inputBindings = [], DE.argsVals = Map.empty } :: IO CS.Item

: 

To fix our engineered failure, we can set the environment variable that our task uses:

In [7]:
setEnv tmpEnvVar "hello there!"
lookupEnv tmpEnvVar

Just "hello there!"

With the environment variable that we've used in our task now set, we can run the flow that wraps our task, and get past the _config-time_ failure associated with the nonexistent environment variable:

In [8]:
runFlow dockFlow DE.DockerTaskInput{ DE.inputBindings = [], DE.argsVals = Map.empty } :: IO CS.Item

: 